In [1]:
pip install pingouin

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 198.9/198.9 kB 2.0 MB/s eta 0:00:00


In [26]:

from scipy.stats import kruskal
import scikit_posthocs as sp
import numpy as np
import pingouin as pg
import seaborn as sns
import matplotlib.pyplot as plt
from scipy.stats import shapiro
from scipy.stats import friedmanchisquare
import pandas as pds
from scipy.stats import kruskal
import pingouin as pg



url = #survey CSV here
df = pd.read_csv(url)
columns_to_check = ['Q1', 'Q2', 'Q3', 'Q4', 'Q5', 'Q6', 'Q7', 'Q8', 'Q10', 'Q11', 'Q12', 'Q13', 'Q14', 'Q16', 'Q17']


In [ ]:

for col in columns_to_check:

    # Shapiro-Wilk test for normality
    stat, p = shapiro(df[col].dropna())  # dropna() is used to remove any NaN values

    if p > 0.05:
        print(f"{col} seems to have a normal distribution (p={p:.2f})")
    else:
        print(f"{col} does not seem to have a normal distribution (p={p:.2f})")

Q1 does not seem to have a normal distribution (p=0.00)
Q2 does not seem to have a normal distribution (p=0.00)
Q3 does not seem to have a normal distribution (p=0.00)
Q4 does not seem to have a normal distribution (p=0.00)
Q5 does not seem to have a normal distribution (p=0.00)
Q6 does not seem to have a normal distribution (p=0.00)
Q7 does not seem to have a normal distribution (p=0.00)
Q8 does not seem to have a normal distribution (p=0.00)
Q10 does not seem to have a normal distribution (p=0.00)
Q11 does not seem to have a normal distribution (p=0.00)
Q12 does not seem to have a normal distribution (p=0.00)
Q13 does not seem to have a normal distribution (p=0.00)
Q14 does not seem to have a normal distribution (p=0.00)
Q16 does not seem to have a normal distribution (p=0.00)
Q17 does not seem to have a normal distribution (p=0.00)


In [ ]:

means = df.groupby('experiment_type')[columns_to_check].mean().T.round(2)
std_devs = df.groupby('experiment_type')[columns_to_check].std().T.round(2)

combined = means.astype(str) + " (" + std_devs.astype(str) + ")"


print("Combined Means and Standard Deviations:\n", combined)


Combined Means and Standard Deviations:
 experiment_type            0            1            2            3  \
Q1               5.64 (1.48)  5.51 (1.56)  5.81 (1.11)  5.92 (1.25)   
Q2               5.21 (1.88)  5.44 (1.63)  5.55 (1.34)  5.65 (1.29)   
Q3               4.85 (2.08)   4.6 (2.01)  5.13 (2.03)  4.86 (1.82)   
Q4                5.6 (1.44)  5.66 (1.71)  5.68 (1.37)   5.88 (1.4)   
Q5               5.52 (1.44)  5.66 (1.27)  5.59 (1.27)  5.65 (1.24)   
Q6               6.38 (1.22)  6.04 (1.36)  6.17 (1.05)   6.3 (1.15)   
Q7                6.3 (1.07)  6.34 (1.15)  6.35 (1.07)   6.5 (0.89)   
Q8               6.33 (1.28)  6.12 (1.31)  6.45 (0.99)  6.26 (1.21)   
Q10              5.94 (1.05)   5.84 (1.3)  5.46 (1.36)  5.65 (1.43)   
Q11              4.44 (1.26)   4.69 (1.3)  4.49 (1.36)  4.41 (1.49)   
Q12              5.65 (1.43)   5.6 (1.52)  5.91 (1.19)  5.84 (1.28)   
Q13              5.54 (1.54)  5.53 (1.41)  5.97 (1.02)  5.98 (1.38)   
Q14              5.72 (1.71)  5.77 (

In [ ]:
p_values_dict = {}

for column in columns_to_check:
    samples = [group[column].dropna().values for _, group in df.groupby('experiment_type')]
    _, p = kruskal(*samples)
    p_values_dict[column] = p



means['p-value'] = pd.Series(p_values_dict)

print(means)

experiment_type     0     1     2     3     4     5   p-value
Q1               5.64  5.51  5.81  5.92  5.50  5.91  0.214190
Q2               5.21  5.44  5.55  5.65  5.54  5.99  0.168404
Q3               4.85  4.60  5.13  4.86  4.58  5.18  0.176207
Q4               5.60  5.66  5.68  5.88  5.79  6.01  0.285408
Q5               5.52  5.66  5.59  5.65  5.39  5.68  0.850081
Q6               6.38  6.04  6.17  6.30  5.98  6.35  0.054971
Q7               6.30  6.34  6.35  6.50  6.36  6.49  0.652556
Q8               6.33  6.12  6.45  6.26  6.34  6.65  0.118194
Q10              5.94  5.84  5.46  5.65  5.66  5.99  0.047373
Q11              4.44  4.69  4.49  4.41  4.72  4.70  0.464500
Q12              5.65  5.60  5.91  5.84  5.62  6.02  0.387362
Q13              5.54  5.53  5.97  5.98  5.76  6.12  0.018212
Q14              5.72  5.77  6.06  5.99  5.75  6.12  0.781203
Q16              6.27  6.23  6.41  6.38  6.39  6.49  0.393729
Q17              6.38  6.27  6.58  6.34  6.50  6.50  0.295850


In [32]:


def posthoc(df, column_name):
    df_column = df[['experiment_type', column_name]].copy()
    median_column = df[column_name].median()
    df_column.loc[:, column_name] = df_column[column_name].fillna(median_column)
    mean_by_experiment_type = df_column.groupby('experiment_type')[column_name].mean()


    print(f"Mean {column_name} by experiment type:")
    print(mean_by_experiment_type)

    experiment_groups = []
    for experiment_type, group_data in df_column.groupby('experiment_type'):
        experiment_groups.append(group_data[column_name].dropna())

    # Run Kruskal-Wallis test
    _, p_value = kruskal(*experiment_groups)
    print(f"Kruskal-Wallis p-value for {column_name}: {p_value}")

    # Run non-parametric pairwise comparisons
    posthoc_comp = pg.pairwise_tests(data=df_column, dv=column_name, between='experiment_type', correction='auto', parametric=False)
    print(f"Pairwise non-parametric tests for {column_name}:\n{posthoc_comp}")




In [33]:
posthoc(df, 'Q6')

Mean Q6 by experiment type:
experiment_type
0    6.382716
1    6.038961
2    6.166667
3    6.300000
4    5.975000
5    6.350000
Name: Q6, dtype: float64
Kruskal-Wallis p-value for Q6: 0.054971014518015844
Pairwise non-parametric tests for Q6:
           Contrast  A  B  Paired  Parametric   U-val alternative     p-unc  \
0   experiment_type  0  1   False       False  3692.5   two-sided  0.023782   
1   experiment_type  0  2   False       False  3711.5   two-sided  0.031327   
2   experiment_type  0  3   False       False  3480.0   two-sided  0.342906   
3   experiment_type  0  4   False       False  3802.0   two-sided  0.031567   
4   experiment_type  0  5   False       False  3277.0   two-sided  0.881421   
5   experiment_type  1  2   False       False  2950.5   two-sided  0.840212   
6   experiment_type  1  3   False       False  2722.0   two-sided  0.164307   
7   experiment_type  1  4   False       False  3026.5   two-sided  0.840155   
8   experiment_type  1  5   False       False 

In [22]:
posthoc(df, 'Q10')

Mean Q10 by experiment type:
experiment_type
0    5.938272
1    5.844156
2    5.461538
3    5.650000
4    5.662500
5    5.987500
Name: Q10, dtype: float64
Kruskal-Wallis p-value for Q10: 0.04737292326556349
Pairwise non-parametric tests for Q10:
           Contrast  A  B  Paired  Parametric   U-val alternative     p-unc  \
0   experiment_type  0  1   False       False  3144.5   two-sided  0.925095   
1   experiment_type  0  2   False       False  3764.5   two-sided  0.029095   
2   experiment_type  0  3   False       False  3497.0   two-sided  0.358808   
3   experiment_type  0  4   False       False  3549.5   two-sided  0.264766   
4   experiment_type  0  5   False       False  2910.5   two-sided  0.234176   
5   experiment_type  1  2   False       False  3530.0   two-sided  0.050160   
6   experiment_type  1  3   False       False  3295.5   two-sided  0.428325   
7   experiment_type  1  4   False       False  3341.0   two-sided  0.334497   
8   experiment_type  1  5   False       Fal

In [34]:
posthoc(df, 'Q13')

Mean Q13 by experiment type:
experiment_type
0    5.543210
1    5.532468
2    5.974359
3    5.975000
4    5.762500
5    6.125000
Name: Q13, dtype: float64
Kruskal-Wallis p-value for Q13: 0.018212412278751044
Pairwise non-parametric tests for Q13:
           Contrast  A  B  Paired  Parametric   U-val alternative     p-unc  \
0   experiment_type  0  1   False       False  3223.5   two-sided  0.707186   
1   experiment_type  0  2   False       False  2800.5   two-sided  0.198346   
2   experiment_type  0  3   False       False  2719.0   two-sided  0.063862   
3   experiment_type  0  4   False       False  2974.5   two-sided  0.349965   
4   experiment_type  0  5   False       False  2557.0   two-sided  0.014566   
5   experiment_type  1  2   False       False  2522.0   two-sided  0.071294   
6   experiment_type  1  3   False       False  2367.0   two-sided  0.008611   
7   experiment_type  1  4   False       False  2673.0   two-sided  0.137385   
8   experiment_type  1  5   False       Fa

In [24]:

def hedges_g(x1, x2):
    """Calculate Hedge's g effect size, including direction."""
    n1, n2 = len(x1), len(x2)
    s1, s2 = np.var(x1, ddof=1), np.var(x2, ddof=1)
    s = np.sqrt(((n1 - 1) * s1 + (n2 - 1) * s2) / (n1 + n2 - 2))
    g = (np.mean(x1) - np.mean(x2)) / s  # Keep the direction
    correction = 1 - (3 / (4*(n1 + n2) - 9))
    g *= correction
    return g

def conover(df, column_name):
    df_column = df[['experiment_type', column_name]].copy()
    median_column = df[column_name].median()
    df_column.loc[:, column_name] = df_column[column_name].fillna(median_column)
    mean_by_experiment_type = df_column.groupby('experiment_type')[column_name].mean()

    print(f"Mean {column_name} by experiment type:")
    print(mean_by_experiment_type)

    experiment_groups = [group[column_name].dropna().values for name, group in df_column.groupby('experiment_type')]

    # Run Kruskal-Wallis test
    _, p_value = kruskal(*experiment_groups)
    print(f"Kruskal-Wallis p-value for {column_name}: {p_value}")

    if p_value < 0.05:
        # Run Conover test for pairwise comparisons
        conover_result = sp.posthoc_conover(df_column, val_col=column_name, group_col='experiment_type', p_adjust=None)


        for i in range(len(experiment_groups)):
            for j in range(i+1, len(experiment_groups)):
                p_val = conover_result.iloc[i, j]
                g_value = hedges_g(experiment_groups[i], experiment_groups[j])
                print(f"Contrast between Group {i} and Group {j}: Conover p-value = {p_val:.6f}, Hedge's g = {g_value:.3f}")
    else:
        print("No significant differences found by Kruskal-Wallis test.")


conover(df, 'Q13')


Mean Q13 by experiment type:
experiment_type
0    5.543210
1    5.532468
2    5.974359
3    5.975000
4    5.762500
5    6.125000
Name: Q13, dtype: float64
Kruskal-Wallis p-value for Q13: 0.018212412278751044
Contrast between Group 0 and Group 1: Conover p-value = 0.570699, Hedge's g = 0.007
Contrast between Group 0 and Group 2: Conover p-value = 0.257646, Hedge's g = -0.327
Contrast between Group 0 and Group 3: Conover p-value = 0.038874, Hedge's g = -0.294
Contrast between Group 0 and Group 4: Conover p-value = 0.318137, Hedge's g = -0.145
Contrast between Group 0 and Group 5: Conover p-value = 0.010823, Hedge's g = -0.424
Contrast between Group 1 and Group 2: Conover p-value = 0.093359, Hedge's g = -0.358
Contrast between Group 1 and Group 3: Conover p-value = 0.009322, Hedge's g = -0.316
Contrast between Group 1 and Group 4: Conover p-value = 0.121239, Hedge's g = -0.158
Contrast between Group 1 and Group 5: Conover p-value = 0.002107, Hedge's g = -0.457
Contrast between Group 2 and